In [18]:
import sys
#!{sys.executable} -m pip install selenium
#!{sys.executable} -m pip install beautifulsoup4
#!{sys.executable} -m pip install Scrapy
#!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install bs4
# selenium va permettre d'ouvrir un navigateur et d'ouvrir le site qui va dynamiquement générer du htlm à partir du javascript

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=6f5d99046f83e13a4ee39d56937388571abe8b854f5cd7b40a76024f21f5d0f0
  Stored in directory: /home/hadoop/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [ ]:
!{sys.executable} -m pip freeze

In [1]:
import os
os.chdir("C:/Users/1992572/Desktop/enseignement/BD2-UBR4120-Bloc1-Hiver2021/cours7/monsterjson/")
os. getcwd()

'C:\\Users\\1992572\\Desktop\\enseignement\\BD2-UBR4120-Bloc1-Hiver2021\\cours7\\monsterjson'

In [19]:
from bs4 import BeautifulSoup
import pandas as pd 
import json

In [20]:
from selenium import webdriver

# download chrome correct package and put it in a specific folder on your harddrive https://www.scrapingbee.com/blog/selenium-python/
DRIVER_PATH = '/usr/local/Downloads/WebDrivers/geckodriver'
driver = webdriver.Firefox(executable_path=DRIVER_PATH)
driver.maximize_window()
driver.get('https://www.monster.com/jobs/search?q=Data+Analyst&where=toronto%2C+on')
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

<ipython-input-20-7919b2ebafe4>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=DRIVER_PATH)


In [21]:
driver.execute_script("window.scrollTo(100, document.body.scrollHeight -100);")

In [22]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
#Après inspection de la page
#results = soup.find(name = "job-results-list")
#print(results)
results = soup.find('div', class_ = "results-list")
job_html = results.find_all('div', class_ = "title-company-location")

rows = []
for job_elem in results:
    title_elem = job_elem.find("h2", title =True)
    company_elem = job_elem.find("h3",attrs={"name":"card_companyname"})
    location_elem = job_elem.find(class_ = "meta-info-sm",attrs={"name":"card_location"})
    link_elem = job_elem.find("a", href=True)
    if None in (title_elem, company_elem, location_elem):
        continue
    #chaque ligne
    row = [title_elem.text.strip(),
    company_elem.text.strip(),
    location_elem.text.strip(),       
    "https://www.monster.com" + link_elem['href'].strip()]

    #append dans ma liste
    rows.append(row)
 
print(head(rows),3)



AttributeError: 'NoneType' object has no attribute 'find_all'

In [5]:
#Création des entetes et insertion dans mon dataframe, ajout des colonnes
from datetime import date #ajoutdechamp
columns = ['Titre','Compagnie','Lieu','Lien']
df = pd.DataFrame(rows, columns=columns)
# df['offre'] = CONST_URL #ajoutdechamp
df['date de la recherche'] = date.today() #ajoutdechamp
print(df)


                                             Titre  \
0                      Data Analyst (Microsoft BI)   
1                                     Data Analyst   
2                     Derivatives Business Analyst   
3                                 Business Analyst   
4                  Data Analyst - Asset Management   
5   2 Business Analysts (Data - Diversity) (73043)   
6                                  Data Specialist   
7       Guidewire Business Analyst - Policy Center   
8                    Intermediate Business Analyst   
9                                 Business Analyst   
10                                    Data Analyst   
11                     Salesforce Business Analyst   
12                             Senior Data Analyst   
13           Intermediate/Senior Business Analysts   
14                      Sr. Business Analyst - P&C   
15                             Sr Business Analyst   
16         Jr. Business Operations Support Analyst   
17                        Bu

In [ ]:
print(soup.prettify())

In [6]:
#sauvegarde en json
jsonFile = df.to_json (orient='records')
with open('scrapping_.json', 'w', encoding='UTF-8') as outfile:
             json.dump(json.loads(jsonFile), outfile, indent=2)

In [7]:
#Push des données dans MongoDB

import pymongo
import json

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["Monster1"]
mycol = mydb["import1"]


with open('scrapping_.json') as file: 
    file_data = json.load(file) 

if isinstance(file_data, list): 
    mycol.insert_many(file_data)   
else: 
    mycol.insert_one(file_data) 